# AT&T Spams detector

## Loading the data and libraries

In [19]:
# librairies usuelles
import pandas as pd
import numpy as np
import plotly.express as px
# librairies qui vont servir lors du preprocessing textuel
import datetime
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import re

# librairies pour nos modèles : Tensorflow et Transformers de Hugging Face (pour les modèles pré-entraînés)
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer

# classes et méthodes de Scikit-Learn pour le préprocessing, le splitting, les métriques, etc...
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score

# On importe également les plugings params pour Tensorboard, pour le tracking des performances de nos modèles
from tensorboard.plugins.hparams import api as hp
import warnings
warnings.filterwarnings('ignore')

In [20]:
# import vocabulary package
# Charger le modèle de langue anglais
!python -m spacy download en_core_web_md


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 27.4 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [21]:
# Load dataset 

data = pd.read_csv('./spam.csv', encoding = 'ISO-8859-1')
print(data.head())

     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  


In [22]:

# Compter le nombre de valeurs NaN pour chaque colonne
nan_counts = data.isna().sum()

print("Nombre de valeurs NaN par colonne :")
print(nan_counts)

Nombre de valeurs NaN par colonne :
v1               0
v2               0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64


In [23]:
# Supprimer les colonnes qui contiennent plus de 50% de valeurs NaN

data = data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)
print(data.head())
print(data.shape)

print("\n Nombre de hams et de span: \n", data['v1'].value_counts())

     v1                                                 v2
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...
(5572, 2)

 Nombre de hams et de span: 
 v1
ham     4825
spam     747
Name: count, dtype: int64


# Preprocessing

## Nettoyage du texte 

In [24]:
# remove poncutuations and convert to lower cases

data['v2_clean'] = data['v2'].apply(lambda s : re.sub(r'[^\w\s]', '', s).lower())
data.head()

,v1,v2,v2_clean
0,ham,"Go until jurong point, crazy.. Available only ...",go until jurong point crazy available only in ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor u c already then say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah i dont think he goes to usf he lives aroun...


## Lemmatisation et tokenisation

In [25]:
# lemmatize 
nlp = spacy.load('en_core_web_md')
data["v2_lemma"] = data["v2_clean"].apply(lambda x: " ".join([token.lemma_ for token in nlp(x) if (token.lemma_ not in STOP_WORDS) and (token.text not in STOP_WORDS)]))    
data.head()


,v1,v2,v2_clean,v2_lemma
0,ham,"Go until jurong point, crazy.. Available only ...",go until jurong point crazy available only in ...,jurong point crazy available bugis n great wor...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni,ok lar joke wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor u c already then say,u dun early hor u c
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah i dont think he goes to usf he lives aroun...,nah think usf live


In [26]:
## tokenisation

# Créer un objet Tokenizer avec un vocabulaire maximal de 2000 mots et un token "out_of_vocab" pour les mots hors vocabulaire
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=2000, oov_token="out_of_vocab")

# Adapter le tokenizer aux textes de la colonne 'v2_lemma' de la variable 'data'
tokenizer.fit_on_texts(data['v2_lemma'])

# Obtenir la taille du vocabulaire (nombre de mots) basée sur le nombre maximal de mots spécifié
vocab_size = tokenizer.num_words
print(vocab_size)
# afficher le mapping entre les indices et les mots du vocabulaire construits à partir de la colonne 'v2_lemma'
# Transformer les textes de la colonne 'v2_lemma' en séquences d'entiers en utilisant le tokenizer
data["v2_tokenized"] = tokenizer.texts_to_sequences(data['v2_lemma'])
print(tokenizer.index_word)

# Afficher les premières lignes du DataFrame 'data' pour visualiser les résultats
print(data.head())


2000
{1: 'out_of_vocab', 2: 'u', 3: 'm', 4: '2', 5: 'ur', 6: 'come', 7: 's', 8: '4', 9: 'know', 10: 'ok', 11: 'ltgt', 12: 'free', 13: 'good', 14: 'send', 15: 'like', 16: 'want', 17: 'day', 18: 'ill', 19: 'time', 20: 'love', 21: 'text', 22: 'tell', 23: 'think', 24: 'need', 25: 'txt', 26: 'home', 27: 'lor', 28: 'reply', 29: 'stop', 30: 'sorry', 31: 'r', 32: 'd', 33: 'win', 34: 'mobile', 35: 'today', 36: 'n', 37: 'phone', 38: 'new', 39: 'week', 40: 'later', 41: 'work', 42: 'ask', 43: 'da', 44: 'hi', 45: 'miss', 46: 'ì', 47: 'hope', 48: 'night', 49: 'claim', 50: 'meet', 51: 'wait', 52: 'thing', 53: 'great', 54: 'try', 55: 'oh', 56: 'leave', 57: 'hey', 58: 'dear', 59: 'pls', 60: 'wat', 61: 'happy', 62: 'message', 63: 'number', 64: 'friend', 65: 'feel', 66: 'thank', 67: 'way', 68: 've', 69: 'late', 70: 'prize', 71: 'right', 72: 'msg', 73: 'find', 74: 'pick', 75: 'tomorrow', 76: 'yes', 77: 'let', 78: 'yeah', 79: 'e', 80: '1', 81: 'babe', 82: 'amp', 83: 'sleep', 84: 'min', 85: 'c', 86: 'life',

In [30]:
print(data.v1.value_counts())

v1
ham     4825
spam     747
Name: count, dtype: int64


In [31]:
#supression de l'étape de padding pour l'ffectuer après l'oversampling / unersampling 
#data_pad = tf.keras.preprocessing.sequence.pad_sequences(data.v2_tokenized, padding="post")

### Ensemble de données d'entrainement / test et de validation

On travaille sur une tâche de classification où les classes sont déséquilibrées (il y a environ 6 fois plus de ham que de spams), donc on utilise la stratification pour s' assurer que la proportion de chaque classe est maintenue dans les ensembles dde train et test.

De plus, les hams représentent 87% des données, alors que les spams représentent seulement 13%. On va donc sélectionner un test size donc on choisit la valeur de l'argument test_size de manière à maintenir la même proportion entre les classes dans l'ensemble de test que dans l'ensemble d'entraînement. Cela garantit que notre modèle est évalué de manière équitable sur toutes les classes

In [23]:
def data_under_sampling(data, nb_reduce):
  data_sample_2 = data.loc[data['v1']=='spam',:]
  len_class_sub = len(data_sample_2)
  # extrait toutes les lignes de l'ensemble de données où la colonne 'v1' a la valeur 'ham'.
  # Ensuite, la méthode .sample() est utilisée pour sous-échantillonner cette classe "ham". 
  # Le nombre d'échantillons de la classe "ham" sous-échantillonnée est déterminé en multipliant nb_reduce par la longueur de la classe "spam" calculée précédemment. 
  # Cela réduit la classe majoritaire à un multiple de la taille de la classe minoritaire.
  data_sample_1 = data.loc[data['v1']=='ham',:].sample(nb_reduce * len_class_sub) 
  # Concatène verticalement (axe 0) les deux sous-ensembles de données résultants (le sous-ensemble de données "ham" sous-échantillonné et 
  # le sous-ensemble de données "spam"). Ensuite, .sample(frac=1) est utilisé pour mélanger aléatoirement les lignes de l'ensemble de données résultant, 
  # et .reset_index(drop=True) réinitialise les index des lignes pour créer un nouvel ensemble de données sous-échantillonné et mélangé.".  
  return pd.concat([data_sample_1,data_sample_2], axis=0).sample(frac=1).reset_index(drop=True)

data_red = data_under_sampling(data, 4)
print(data_red.head())
print(data_red.shape)
print(data_red['v1'].value_counts())

<class 'numpy.float64'>


In [ ]:
# split train test sets with 0.13 due to 
data_train_test, data_val = train_test_split(data_red, test_size=0.13, stratify=data['v1'], random_state=2)
print(data_train_test)
print(data_val)

In [24]:
train = tf.data.Dataset.from_tensor_slices((xtrain, ytrain))
val = tf.data.Dataset.from_tensor_slices((xval, yval))

In [25]:
train_batch = train.shuffle(len(train)).batch(64)
val_batch = val.shuffle(len(val)).batch(64)

## Modeling

Let's create a model in order to train an embedding!

1. Follow a similar architecture to the one we used in the code embedding demonstration.

In [26]:
model = tf.keras.Sequential([
                  # Couche d'Input Word Embedding           
                  tf.keras.layers.Embedding(vocab_size+1, 8, input_shape=[data[['v1']].shape[1],],name="embedding"),
                  # Gobal average pooling
                  tf.keras.layers.GlobalAveragePooling1D(),

                  # Couche Dense classique
                  tf.keras.layers.Dense(16, activation='relu'),

                  # Couche de sortie avec le nombre de neurones en sortie égale au nombre de classe avec fonction softmax
                  tf.keras.layers.Dense(1, activation="linear")
])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 8)              16008     
                                                                 
 global_average_pooling1d (  (None, 8)                 0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 16)                144       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 16169 (63.16 KB)
Trainable params: 16169 (63.16 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
optimizer= tf.keras.optimizers.Adam()

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=[tf.keras.metrics.MeanAbsoluteError()])

In [28]:
history = model.fit(train_batch, 
                    epochs=20, 
                    validation_data=val_batch)

Epoch 1/20


: 